In [1]:
import glob, re, nltk
from collections import Counter

In [2]:
single_term_idx = Counter()
#read stop words
stop_words = set()
f = open('stops.txt', 'r')
for line in f:
    stop_words.add(line.strip())
f.close()

In [3]:
def removeHyphen(match):
    return match.group().replace('-', '')

def hyphenReplace(line, regex, docno):
    words = regex.finditer(line)
    if words:
        for i in words:
            word = i.group()
            li = word.split('-')
            for item in li:
                if not item in stop_words:
                    single_term_idx[(item, docno)] += 1
        line = regex.sub(removeHyphen, line)
    return line

def prefixReplace(line, regex, docno):
    words = regex.finditer(line)
    if words:
        for word in words:
            stem = word.group(2)
            if not stem in stop_words:
                single_term_idx[(stem, docno)] += 1
        line = regex.sub(r'\g<1>\g<2>', line)
    return line

In [4]:
def writeToFile(index_table):
    # write to output
    output_file = open('single_term_idx_table.txt', 'w')
    for key in sorted(index_table.keys()):
        output_file.write(' '.join(str(x) for x in key)+' '+str(index_table[key])+'\n')
    output_file.close()

def createIndexTable(input_path):
    # format input_path to ./input_path/
    if input_path[0] == '/':
        input_path = '.' + input_path
    if input_path[-1] != '/':
        input_path += '/'

    files = glob.glob(input_path+'*') # grab all the files under path
    get_num = re.compile(r'(\d+)')
    p_prefix = re.compile(r'\b(a|an|ante|anti|auto|circum|co|com|con|contra|contro|de|dis|en|em|ex|extra|fore|hetero|homo|homeo|hyper|il|im|in|ir|inter|intra|intro|macro|micro|mid|mis|mono|non|omni|over|post|pre|pro|re|semi|sub|super|sym|syn|trans|tri|un|under|uni)-([a-z])+\b', re.I)
    p_hyphen = re.compile(r'\b(\w+-)+\w+\b')
    
    for file in files:
        f = open(file, 'r')
        docno = int(get_num.search(file).group(1))
        for line in f:
            line = line.strip()
            line = line.lower()
            # grep prefix
            line = prefixReplace(line, p_prefix, docno)
            # grep hyphenated word
            line = hyphenReplace(line, p_hyphen, docno)
            line = line.replace(':', ' ')
            li = nltk.word_tokenize(line)
            for word in li:
                if not word in stop_words:
                    # build single term idx
                    single_term_idx[(word, docno)] += 1

createIndexTable('data/tags_test')
# writeToFile(single_term_idx)
